In [4]:
import os
import time
import yaml
import torch
from botorch.models import SingleTaskGP
from botorch.models.transforms import Normalize, Standardize
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import LogExpectedImprovement
from botorch.optim import optimize_acqf

import darkfield.mmmUtils_v2 as mu
from bash_config import write_bash

# Constants
N_SIM_POINTS = 2000

####### Parameters to scan ########

PARAMS = {
    'O2.size': [0.0, 340.0],
    'O1.size': [0.0, 340.0],
    'A1.size': [0.0, 340.0],
    'A2.size': [0.0, 340.0],
    'O1.position': [-10.0, 0.0],
}


param_names = list(PARAMS.keys())
BOUNDS = torch.tensor([PARAMS[name] for name in param_names], dtype=torch.float64).T
BOUNDS_NORMALIZED = torch.tensor([[0.0] * len(param_names), [1.0] * len(param_names)], dtype=torch.float64)

#BOUNDS = torch.tensor([[0.0], [340.0]], dtype=torch.float64)  # microns
#BOUNDS_NORMALIZED = torch.tensor([[0.0], [1.0]], dtype=torch.float64)

print(BOUNDS.shape)

torch.Size([2, 5])


In [2]:


def generate_yaml(template_path, output_path, param_values, base_index=1):
    with open(template_path) as f:
        ip = yaml.safe_load(f)

    for name, val in zip(param_names, param_values):
        keys = name.split('.')  # e.g., ['O2', 'size']
        d = ip
        for k in keys[:-1]:
            d = d[k]
        if 'size' in name or 'amplitude' in name or 'lambda' in name:
            d[keys[-1]] = float(val * 1e-6)  # convert microns → meters
        else:
            d[keys[-1]] = float(val)

    # Automatically enable O2, O1, etc., if their size > 0
    for obj in ['O1', 'O2', 'A1', 'A2', 'PH']:
        if f'{obj}.size' in param_names:
            i = param_names.index(f'{obj}.size')
            ip[obj]['in'] = 0 if param_values[i] == 0 else 1

    filename = f"BO_{base_index}.yaml"
    fullpath = os.path.join(output_path, filename)
    with open(fullpath, 'w') as f_out:
        yaml.dump(ip, f_out, sort_keys=False)
    return filename, fullpath


def submit_job(yaml_filename, sim_index, n_sim_points=N_SIM_POINTS, n_cpus=24, mem='600GB'):
    bash_dir = '/home/yu79deg/darkfield_p5438/bash'
    bash_path = write_bash(path=bash_dir, N=n_sim_points, upd_params={'n_cpus': n_cpus, 'mem': mem, 'yaml': yaml_filename})
    os.system(f'sbatch {bash_path}')

def wait_for_completion(jobname, timeout=3600, check_interval=20):
    result_path = f'/home/yu79deg/darkfield_p5438/Aime/pickles/{jobname}_res.pickle'
    waited = 0
    while not os.path.exists(result_path) and waited < timeout:
        print(f'Waiting for job {jobname} to complete...')
        time.sleep(check_interval)
        waited += check_interval
    if not os.path.exists(result_path):
        raise TimeoutError(f'Job {jobname} did not complete in time.')
    return result_path

def extract_shadow_factor(pickle_path):
    result_data = mu.loadPickle(pickle_path)
    params = result_data[1]
    shadow = params['intensities']['roi2'] / params['intensities']['TCC']
    return shadow

def objective_function(x_tensor, template_path, output_path, base_index):
    #x_tensor : list of (normalised) input params to vary
    x_denorm = x_tensor * (BOUNDS[1] - BOUNDS[0]) + BOUNDS[0]
    param_values = x_denorm.tolist()[0]

    yaml_filename, _ = generate_yaml(template_path, output_path, param_values, base_index)
    jobname = os.path.splitext(yaml_filename)[0]
    submit_job(yaml_filename, sim_index=base_index)
    print(f"Running: {dict(zip(param_names, param_values))}")
    result_path = wait_for_completion(jobname)
    shadow_factor = extract_shadow_factor(result_path)
    return -torch.log10(torch.tensor([[shadow_factor]], dtype=torch.float64))

def bayesian_optimization(n_iters=10, n_init=3):
    template_path = '/home/yu79deg/darkfield_p5438/yamls/BO_template.yaml'
    output_path = '/home/yu79deg/darkfield_p5438/yamls'
    dim = len(param_names)

    train_x = torch.rand(n_init, dim, dtype=torch.float64)
    train_y = torch.vstack([
        objective_function(train_x[i:i+1], template_path, output_path, i + 1)
        for i in range(n_init)
    ])

    for iteration in range(n_init, n_iters):
        model = SingleTaskGP(
            train_X=train_x,
            train_Y=train_y,
            input_transform=Normalize(d=dim),
            outcome_transform=Standardize(m=1)
        )
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_mll(mll)

        logEI = LogExpectedImprovement(model=model, best_f=train_y.max())
        candidate, _ = optimize_acqf(
            logEI, bounds=BOUNDS_NORMALIZED, q=1, num_restarts=5, raw_samples=20
        )

        new_x = candidate.detach()
        new_y = objective_function(new_x, template_path, output_path, iteration + 1)

        train_x = torch.cat([train_x, new_x])
        train_y = torch.cat([train_y, new_y])

    return train_x, train_y


In [ ]:
results_x, results_y = bayesian_optimization(n_iters=70, n_init=10)

Submitted batch job 3875624
Running: {'O2.size': 126.68178761077453, 'O1.size': 231.3831771249462, 'A1.size': 109.7835657905362, 'A2.size': 118.4016526179966, 'O1.position': -0.7355927970090708}
Waiting for job BO_1 to complete...
Waiting for job BO_1 to complete...
Waiting for job BO_1 to complete...


In [5]:
import pandas as pd
import numpy as np

# Denormalize parameter values
x_vals = results_x * (BOUNDS[1] - BOUNDS[0]) + BOUNDS[0]
real_shadow = (10 ** (-results_y)).numpy().flatten()

# Format shadow factor as "a × 10^b"
def format_sci(val):
    if val == 0:
        return "0"
    exponent = int(np.floor(np.log10(val)))
    base = val / (10 ** exponent)
    return f"{base:.1f} × 10^{exponent}"

shadow_strs = [format_sci(val) for val in real_shadow]

# Create DataFrame with only formatted shadow
DataFrame = pd.DataFrame(x_vals.numpy(), columns=param_names)
DataFrame['Shadow_factor'] = shadow_strs

# Find the best and next 4 best values
sorted_idx = np.argsort(real_shadow)
best_idx = sorted_idx[0]
next4_idx = sorted_idx[1:5]

# Styling function
def highlight_best_formatted(val, idx):
    if idx == best_idx:
        return 'background-color: orange'
    elif idx in next4_idx:
        return 'background-color: yellow'
    else:
        return ''

# Apply styling only to the 'Shadow_factor' column
styled_df = DataFrame.style.apply(
    lambda col: [highlight_best_formatted(v, i) if col.name == 'Shadow_factor' else '' for i, v in enumerate(col)],
    axis=0)

# Print the table
styled_df


,O2.size,O1.size,A1.size,A2.size,O1.position,Shadow_factor
0,126.681788,231.383177,109.783566,118.401653,-0.735593,3.6 × 10^-7
1,23.002208,317.547550,58.136406,338.068338,-1.213959,4.0 × 10^-9
2,105.198887,330.303427,281.461127,101.431389,-5.945459,6.2 × 10^-4
3,111.427058,49.397464,319.531373,248.471849,-0.489363,3.0 × 10^-1
4,295.365824,319.552989,150.136637,54.889521,-9.233148,3.8 × 10^-7
5,87.567033,148.652479,234.653630,48.523918,-0.340856,1.1 × 10^-7
6,201.243001,237.547647,318.520398,3.867478,-1.874897,3.0 × 10^-8
7,135.499437,12.118478,232.218263,82.666589,-1.715976,3.2 × 10^-5
8,293.916308,156.468571,40.753435,192.728558,-4.785618,2.1 × 10^-7
9,117.787478,51.895450,201.288400,225.396924,-4.499703,1.5 × 10^-2


In [6]:
 390/ (2*0.428)

455.60747663551405